In [23]:
import sqlite3
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score

DB_PATH = "../mlb_scrape.sqlite"
START_YEAR, END_YEAR = 2015, 2025
TEST_START_SEASON, TEST_END_SEASON = 2024, 2025

START_BANKROLL = 5000.0
KELLY_FRACTION = 0.25
MAX_BET_FRAC   = 0.02
MIN_EDGE       = 0.015  # set 0.01 later if you want fewer bets

selected_cols_final = [
 'season','park_pf_runs','diff_slg_x_sp_left','diff_sp_left',
 'diff_bat_season_B1_AB_mean','diff_bat_season_B2_H_mean','diff_bat_season_B3_BB_mean',
 'diff_bat_season_B4_SO_mean','diff_bat_season_B7_OBP_mean','diff_bat_season_B8_SLG_mean',
 'diff_bat_season_B10_Pit_mean','diff_bat_season_B11_Str_mean','diff_bat_season_B12_PO_mean',
 'diff_bat_season_B13_A_mean','diff_bat_last10_B1_AB_mean','diff_bat_last10_B2_H_mean',
 'diff_bat_last10_B4_SO_mean','diff_bat_last10_B10_Pit_mean','diff_bat_last10_B11_Str_mean',
 'diff_bat_last10_B13_A_mean','diff_sp_last3_SP1_IP_mean','diff_sp_last3_SP2_H_mean',
 'diff_sp_last3_SP3_BB_mean','diff_sp_last3_SP4_SO_mean','diff_sp_last3_SP5_HR_mean',
 'diff_sp_last3_SP7_BF_mean','diff_sp_last3_SP9_Str_mean','diff_sp_last3_WHIP_mean',
 'diff_rp_season_P2_H_mean','diff_rp_season_P3_BB_mean','diff_rp_season_P4_SO_mean',
 'diff_rp_season_P5_HR_mean','diff_rp_season_P6_ERA_mean','diff_rp_season_P7_BF_mean',
 'diff_rp_season_P17_IR_mean','diff_rp_season_P18_IS_mean','diff_sp_career_SP1_IP',
 'diff_sp_career_SP3_BB','diff_sp_career_SP6_ERA','diff_sp_career_WHIP'
]



In [24]:
conn = sqlite3.connect(DB_PATH)

existing = set(r[0] for r in conn.execute(
    "SELECT name FROM sqlite_master WHERE type='table' AND name LIKE 'games_table_%'"
).fetchall())

tables = [f"games_table_{y}" for y in range(START_YEAR, END_YEAR + 1) if f"games_table_{y}" in existing]
print("Found tables:", tables)

union_sql = "\nUNION ALL\n".join([f"SELECT * FROM {t}" for t in tables])
df_all = pd.read_sql(f"SELECT * FROM ({union_sql}) ORDER BY gameDate ASC", conn)
print("Combined rows:", len(df_all))

# Parse date
df_all["gameDate_dt"] = pd.to_datetime(df_all["gameDate"], utc=True, errors="coerce")

# Apr 7 cutoff (same as your main notebook)
cutoff_month, cutoff_day = 4, 7
mask_cutoff = ~(
    (df_all["gameDate_dt"].dt.month < cutoff_month) |
    ((df_all["gameDate_dt"].dt.month == cutoff_month) & (df_all["gameDate_dt"].dt.day < cutoff_day))
)
df_all = df_all.loc[mask_cutoff].copy()
print("Rows after Apr 7 cutoff:", len(df_all))

# Pull context
ctx = pd.read_sql("""
    SELECT gamePk, park_pf_runs, home_sp_throws, away_sp_throws
    FROM context_game
""", conn)

df_all = df_all.merge(ctx, on="gamePk", how="left")

# Handedness
df_all["home_sp_left"] = (df_all["home_sp_throws"].astype(str).str.upper() == "L").astype(int)
df_all["away_sp_left"] = (df_all["away_sp_throws"].astype(str).str.upper() == "L").astype(int)
df_all["diff_sp_left"] = df_all["home_sp_left"] - df_all["away_sp_left"]

# SLG x SP-left proxy (uses your existing last10 SLG columns)
df_all["home_slg_x_away_sp_left"] = df_all["home_bat_last10_B8_SLG_mean"] * df_all["away_sp_left"]
df_all["away_slg_x_home_sp_left"] = df_all["away_bat_last10_B8_SLG_mean"] * df_all["home_sp_left"]
df_all["diff_slg_x_sp_left"] = df_all["home_slg_x_away_sp_left"] - df_all["away_slg_x_home_sp_left"]

print("park_pf_runs non-null:", df_all["park_pf_runs"].notna().sum(), "/", len(df_all))


Found tables: ['games_table_2015', 'games_table_2016', 'games_table_2017', 'games_table_2018', 'games_table_2019', 'games_table_2020', 'games_table_2021', 'games_table_2022', 'games_table_2023', 'games_table_2024', 'games_table_2025']
Combined rows: 25193
Rows after Apr 7 cutoff: 24437
park_pf_runs non-null: 24086 / 24437


In [25]:
df_model = df_all.copy()

# Keep these for backtest joins and chronological split
ID_COLS = ["gamePk", "season", "gameDate_dt", "homeWin"]

df_diff = df_model[ID_COLS + ["park_pf_runs", "diff_sp_left", "diff_slg_x_sp_left"]].copy()

home_cols = [c for c in df_model.columns if c.startswith("home_")]
away_cols = [c for c in df_model.columns if c.startswith("away_")]

pairs = {}
away_set = set(away_cols)
for h in home_cols:
    base = h.replace("home_", "")
    a = "away_" + base
    if a in away_set:
        pairs[base] = (h, a)

skipped, diffed = 0, 0
for base, (h, a) in pairs.items():
    h_num = pd.to_numeric(df_model[h], errors="coerce")
    a_num = pd.to_numeric(df_model[a], errors="coerce")
    if h_num.notna().sum() == 0 or a_num.notna().sum() == 0:
        skipped += 1
        continue
    df_diff[f"diff_{base}"] = h_num - a_num
    diffed += 1

print("Paired:", len(pairs), "| diffed numeric:", diffed, "| skipped:", skipped)
print("df_diff shape:", df_diff.shape)
df_diff.head()


Paired: 176 | diffed numeric: 175 | skipped: 1
df_diff shape: (24437, 181)


/var/folders/cf/4zyl4dsd16n9yhd_dkhk66gc0000gn/T/ipykernel_569/2200149294.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_diff[f"diff_{base}"] = h_num - a_num
/var/folders/cf/4zyl4dsd16n9yhd_dkhk66gc0000gn/T/ipykernel_569/2200149294.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_diff[f"diff_{base}"] = h_num - a_num
/var/folders/cf/4zyl4dsd16n9yhd_dkhk66gc0000gn/T/ipykernel_569/2200149294.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, w

,gamePk,season,gameDate_dt,homeWin,park_pf_runs,diff_sp_left,diff_slg_x_sp_left,diff_bat_season_B1_AB_mean,diff_bat_season_B1_AB_std,diff_bat_season_B2_H_mean,...,diff_sp_career_SP1_IP,diff_sp_career_SP2_H,diff_sp_career_SP3_BB,diff_sp_career_SP4_SO,diff_sp_career_SP5_HR,diff_sp_career_SP6_ERA,diff_sp_career_SP7_BF,diff_sp_career_SP8_Pit,diff_sp_career_SP9_Str,diff_sp_career_WHIP
0,413662,2015,2015-04-07 02:05:00+00:00,1.0,100.373,0,0.0,0.0,0.0,0.0,...,101.333333,-95.0,-89.0,341.0,-13.0,-0.48,159.0,0.0,0.0,-0.17
1,413660,2015,2015-04-07 02:10:00+00:00,0.0,107.453,-1,-1.0,0.0,0.0,0.0,...,-1514.000000,-1342.0,-376.0,-1576.0,-170.0,0.17,-6240.0,0.0,0.0,0.05
2,413665,2015,2015-04-07 23:10:00+00:00,0.0,91.084,-1,-1.0,-2.0,0.0,2.0,...,-104.666667,-169.0,-21.0,-162.0,-11.0,-0.14,-510.0,0.0,0.0,-0.05
3,413666,2015,2015-04-07 23:10:00+00:00,0.0,90.310,-1,-1.0,-2.0,0.0,0.0,...,-748.666667,-791.0,-143.0,-520.0,-109.0,0.12,-3124.0,0.0,0.0,0.07
4,413667,2015,2015-04-08 00:10:00+00:00,0.0,106.246,0,0.0,-9.0,0.0,-8.0,...,201.666667,55.0,88.0,235.0,-39.0,-1.13,727.0,0.0,0.0,-0.08


In [26]:
odds = pd.read_sql("""
    SELECT gamePk, home_odds, away_odds
    FROM schedule_games
    WHERE season IN (2024, 2025)
""", conn)

conn.close()

df_diff = df_diff.merge(odds, on="gamePk", how="left")

print("Odds coverage in df_diff test seasons:",
      df_diff.loc[df_diff["season"].isin([2024, 2025]), ["home_odds","away_odds"]].notna().all(axis=1).mean())


Odds coverage in df_diff test seasons: 0.7241830065359477


In [27]:
train = df_diff["season"] < TEST_START_SEASON
test  = df_diff["season"].between(TEST_START_SEASON, TEST_END_SEASON)

df_train = df_diff.loc[train].sort_values("gameDate_dt").copy()
df_test  = df_diff.loc[test].sort_values("gameDate_dt").copy()

Xtr = df_train[selected_cols_final]
ytr = df_train["homeWin"].astype(int)

Xte = df_test[selected_cols_final]
yte = df_test["homeWin"].astype(int)

logreg = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(penalty="l2", C=1.0, solver="lbfgs", max_iter=2000))
])

logreg.fit(Xtr, ytr)
p_home = logreg.predict_proba(Xte)[:, 1]

print("Test AUC:", round(roc_auc_score(yte, p_home), 4))
print("Test ACC:", round(accuracy_score(yte, p_home > 0.5), 4))

df_test = df_test.copy()

def logit(p, eps=1e-6):
    p = np.clip(p, eps, 1 - eps)
    return np.log(p / (1 - p))

# Base model predictions
p_train_raw = logreg.predict_proba(Xtr)[:, 1]

# Fit Platt calibrator on TRAIN ONLY
X_platt = logit(p_train_raw).reshape(-1, 1)
y_platt = ytr.values

platt = LogisticRegression(solver="lbfgs")
platt.fit(X_platt, y_platt)

print("Platt calibration fitted.")

# Raw test probabilities
p_test_raw = p_home

# Calibrated probabilities
p_test_cal = platt.predict_proba(
    logit(p_test_raw).reshape(-1, 1)
)[:, 1]

df_test["p_home"] = p_test_cal



Test AUC: 0.6016
Test ACC: 0.571
Platt calibration fitted.


In [28]:
print("Raw test prob mean:", p_test_raw.mean())
print("Cal test prob mean:", p_test_cal.mean())
print("Raw > 0.5:", (p_test_raw > 0.5).mean())
print("Cal > 0.5:", (p_test_cal > 0.5).mean())

Raw test prob mean: 0.5289190232637341
Cal test prob mean: 0.5288831566365135
Raw > 0.5: 0.6444444444444445
Cal > 0.5: 0.6450980392156863


In [29]:
def american_to_decimal(odds):
    if odds is None:
        return np.nan
    try:
        o = float(odds)
    except Exception:
        return np.nan

    if o == 0:
        return np.nan  # invalid odds

    if o > 0:
        return 1.0 + (o / 100.0)
    else:
        return 1.0 + (100.0 / abs(o))


def implied_prob_from_american(odds):
    dec = american_to_decimal(odds)
    return 1.0 / dec

def kelly_fraction_from_decimal(p, dec):
    b = dec - 1.0
    f = (b * p - (1.0 - p)) / b
    return max(0.0, f)

def edge_allowed(edge):
    return (edge >= 0.05) or (0.015 <= edge <= 0.02)

def pick_best_side(p_home, home_odds, away_odds):
    dec_home = american_to_decimal(home_odds)
    dec_away = american_to_decimal(away_odds)

    # Bail if odds invalid
    if not np.isfinite(dec_home) or not np.isfinite(dec_away):
        return None

    f_home = kelly_fraction_from_decimal(p_home, dec_home)
    f_away = kelly_fraction_from_decimal(1.0 - p_home, dec_away)

    edge_home = p_home - implied_prob_from_american(home_odds)
    edge_away = (1.0 - p_home) - implied_prob_from_american(away_odds)

    best = None
    if edge_allowed(edge_home) and f_home > 0:
        best = ("HOME", p_home, dec_home, f_home, edge_home)

    if edge_allowed(edge_away) and f_away > 0:
        cand = ("AWAY", 1.0 - p_home, dec_away, f_away, edge_away)
        if best is None or cand[3] > best[3]:
            best = cand

    return best

# Only bet games with odds
df_bet = df_test.dropna(subset=["home_odds", "away_odds"]).copy()

bankroll = START_BANKROLL
rows = []

for r in df_bet.itertuples(index=False):
    pick = pick_best_side(r.p_home, r.home_odds, r.away_odds)
    if pick is None:
        continue

    side, p, dec, f_full, edge = pick
    f = min(MAX_BET_FRAC, KELLY_FRACTION * f_full)
    if f <= 0:
        continue

    stake = bankroll * f
    home_won = int(r.homeWin) == 1
    bet_won = home_won if side == "HOME" else (not home_won)
    profit = stake * (dec - 1.0) if bet_won else -stake
    bankroll += profit

    rows.append({
        "gamePk": r.gamePk,
        "season": r.season,
        "date": r.gameDate_dt,
        "side": side,
        "p": float(p),
        "edge": float(edge),
        "odds_home": float(r.home_odds),
        "odds_away": float(r.away_odds),
        "stake": float(stake),
        "profit": float(profit),
        "bankroll": float(bankroll),
    })

bt = pd.DataFrame(rows).sort_values("date")

bt["edge_bin"] = pd.cut(
    bt["edge"],
    bins=[0.015, 0.02, 0.03, 0.05, 1.0],
    labels=["1.5–2%", "2–3%", "3–5%", "5%+"]
)

summary = bt.groupby("edge_bin").agg(
    bets=("edge", "count"),
    roi=("profit", lambda x: x.sum() / bt.loc[x.index, "stake"].sum()),
    win_rate=("profit", lambda x: (x > 0).mean())
)

display(summary)

print("Bets placed:", len(bt))
if len(bt):
    total_profit = bt["profit"].sum()
    total_stake  = bt["stake"].sum()
    roi = total_profit / total_stake if total_stake > 0 else np.nan
    win_rate = (bt["profit"] > 0).mean()

    peak = bt["bankroll"].cummax()
    dd = (bt["bankroll"] - peak) / peak
    max_dd = dd.min()

    print(f"Start bankroll: {START_BANKROLL:,.2f}")
    print(f"End bankroll:   {bankroll:,.2f}")
    # print(f"Total profit:   {total_profit:,.2f}")
    # print(f"Total staked:   {total_stake:,.2f}")
    print(f"ROI:            {roi*100:,.2f}%")
    print(f"Win rate:       {win_rate*100:,.2f}%")
    print(f"Max drawdown:   {max_dd*100:,.2f}%")

bt.head(100)


/var/folders/cf/4zyl4dsd16n9yhd_dkhk66gc0000gn/T/ipykernel_569/1623548635.py:99: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = bt.groupby("edge_bin").agg(
/var/folders/cf/4zyl4dsd16n9yhd_dkhk66gc0000gn/T/ipykernel_569/1623548635.py:101: RuntimeWarning: invalid value encountered in scalar divide
  roi=("profit", lambda x: x.sum() / bt.loc[x.index, "stake"].sum()),


,bets,roi,win_rate
edge_bin,,,
1.5–2%,137,0.050738,0.510949
2–3%,0,NaN,NaN
3–5%,0,NaN,NaN
5%+,1053,0.116736,0.499525


Bets placed: 1190
Start bankroll: 5,000.00
End bankroll:   68,134.22
ROI:            11.30%
Win rate:       50.08%
Max drawdown:   -31.23%


,gamePk,season,date,side,p,edge,odds_home,odds_away,stake,profit,bankroll,edge_bin
0,745523,2024,2024-04-07 17:35:00+00:00,HOME,0.518883,0.018883,100.0,-118.0,47.208468,47.208468,5047.208468,1.5–2%
1,747141,2024,2024-04-07 17:35:00+00:00,AWAY,0.429168,0.106588,-255.0,210.0,100.944169,-100.944169,4946.264299,5%+
2,746734,2024,2024-04-07 17:40:00+00:00,HOME,0.587928,0.092879,102.0,-120.0,98.925286,-98.925286,4847.339013,5%+
3,746330,2024,2024-04-07 18:10:00+00:00,AWAY,0.546630,0.065861,-126.0,108.0,96.946780,-96.946780,4750.392233,5%+
4,745198,2024,2024-04-07 18:15:00+00:00,HOME,0.709033,0.122256,-142.0,120.0,95.007845,-95.007845,4655.384388,5%+
...,...,...,...,...,...,...,...,...,...,...,...,...
95,746078,2024,2024-04-26 23:10:00+00:00,AWAY,0.719170,0.242979,-130.0,110.0,107.526264,118.278890,5494.592065,5%+
96,746808,2024,2024-04-26 23:40:00+00:00,HOME,0.474844,0.130016,190.0,-230.0,109.891841,208.794498,5703.386564,5%+
97,745269,2024,2024-04-27 01:40:00+00:00,AWAY,0.550294,0.119259,-156.0,132.0,114.067731,-114.067731,5589.318833,5%+
98,746075,2024,2024-04-27 20:10:00+00:00,AWAY,0.513251,0.066823,-146.0,124.0,111.786377,138.615107,5727.933940,5%+


In [30]:
bt = bt.copy()

bt["bet_odds"] = np.where(
    bt["side"] == "HOME",
    bt["odds_home"],
    bt["odds_away"]
)

bt["bet_type"] = np.where(
    bt["bet_odds"] < 0,
    "Favorite",
    "Underdog"
)

bt[["side", "bet_odds", "bet_type"]].head()

fav_ud_summary = bt.groupby("bet_type").agg(
    bets=("profit", "count"),
    total_profit=("profit", "sum"),
    total_staked=("stake", "sum"),
    roi=("profit", lambda x: x.sum() / bt.loc[x.index, "stake"].sum()),
    win_rate=("profit", lambda x: (x > 0).mean()),
    avg_odds=("bet_odds", "mean"),
)

fav_ud_summary

,bets,total_profit,total_staked,roi,win_rate,avg_odds
bet_type,,,,,,
Favorite,321,23029.906279,145547.717583,0.158229,0.647975,-133.205607
Underdog,869,40104.317689,413174.007426,0.097064,0.446490,192.983890


In [31]:
def max_drawdown(series):
    peak = series.cummax()
    dd = (series - peak) / peak
    return dd.min()

dd_by_type = (
    bt.sort_values("date")
      .groupby("bet_type")["bankroll"]
      .apply(max_drawdown)
)

dd_by_type


bet_type
Favorite   -0.301105
Underdog   -0.312151
Name: bankroll, dtype: float64